In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
#import prediction
#from utils import load_data,get_data_day
import logging
logging.basicConfig(level=logging.INFO)
import utils
import time

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import pickle
from utils import load_data
from utils import load_data,get_data_day
import numpy as np
import pickle
import datetime
import calendar
import time
import os

import utils
import pandas as pd
import logging

In [2]:
ddd=load_data()

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


In [35]:
sites=ddd['sites']

In [37]:
sites['idPolair']=pd.to_numeric(sites['idPolair'])
sites=sites.set_index('idPolair')

KeyError: 'idPolair'

In [38]:
sites.head()

,nom_station,coord_x_l93,coord_y_l93,X_lamb2,Y_lamb2,LON,LAT,Département,Zone_EPCI,typologie,...,PM10_2015,PM10_2016,PM10_2017,PM25_influence,PM25_2012,PM25_2013,PM25_2014,PM25_2015,PM25_2016,PM25_2017
idPolair,,,,,,,,,,,,,,,,,,,,,
15013,Champ_sur_Drac,914668.44,6445968.5,867090.82,2014539.48,5.728659,45.079595,Isère,CC Sud Grenoblois,Peri-Urbaine,...,NaN,NaN,NaN,NA,NaN,NaN,NaN,NaN,NaN,NaN
15017,Fontaine_les_Balmes,910970.88,6458165.5,863285.49,2026715.96,5.686970,45.190494,Isère,CA Grenoble,Urbaine,...,1.0,1.0,1.0,NA,NaN,NaN,NaN,NaN,NaN,NaN
15018,Voiron_Urbain,902699.00,6476726.0,854847.00,2045221.00,5.589443,45.360014,Isère,CA Pays Voironnais,Urbaine,...,1.0,1.0,NaN,NA,NaN,NaN,NaN,NaN,NaN,NaN
15031,Ecrins,973315.81,6439213.0,925852.02,2008282.66,6.469501,44.998096,Isère,CC du Briançonnais,Rurale,...,NaN,NaN,NaN,NA,NaN,NaN,NaN,NaN,NaN,NaN
15038,Saint_Martin_Heres,916202.00,6457516.0,868526.98,2026110.36,5.753253,45.183027,Isère,CA Grenoble,Urbaine,...,1.0,1.0,1.0,NA,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
sites=sites.iloc[:,1:3]

In [40]:
sites=sites

In [41]:
sites.head()

,coord_x_l93,coord_y_l93
idPolair,,
15013,914668.44,6445968.5
15017,910970.88,6458165.5
15018,902699.00,6476726.0
15031,973315.81,6439213.0
15038,916202.00,6457516.0


In [14]:
data=pd.read_csv('fillna/final.csv')
data['date']=pd.to_datetime(data['date'])
data=data.set_index(['idPolair','date'])

In [15]:
data.head()

O3   NO2       PM10       PM25
idPolair date                                                 
7001.0   2012-01-01 00:00:00  51.0   4.0   3.726041  12.212574
         2012-01-01 01:00:00  53.0   3.0  11.399355  12.212574
         2012-01-01 02:00:00  39.0  10.0  10.017576  12.212574
         2012-01-01 03:00:00  39.0   5.0   7.250695  12.212574
         2012-01-01 04:00:00  30.0   8.0  35.000000  12.212574

In [16]:
newData=pd.merge(sites,data.reset_index().set_index('idPolair'),left_index=True,right_index=True,how='left')

In [17]:
newData.head()

,coord_x_l93,coord_y_l93,date,O3,NO2,PM10,PM25
idPolair,,,,,,,
7001,70.366789,651.834092,2012-01-01 00:00:00,51.0,4.0,3.726041,12.212574
7001,70.366789,651.834092,2012-01-01 01:00:00,53.0,3.0,11.399355,12.212574
7001,70.366789,651.834092,2012-01-01 02:00:00,39.0,10.0,10.017576,12.212574
7001,70.366789,651.834092,2012-01-01 03:00:00,39.0,5.0,7.250695,12.212574
7001,70.366789,651.834092,2012-01-01 04:00:00,30.0,8.0,35.000000,12.212574


In [18]:
newData=newData.reset_index().set_index(['idPolair','date'])

In [19]:
newData.corr()

,coord_x_l93,coord_y_l93,O3,NO2,PM10,PM25
coord_x_l93,1.000000,-0.039298,-0.062872,0.069309,0.167238,0.067437
coord_y_l93,-0.039298,1.000000,-0.065233,0.000950,-0.018252,-0.017690
O3,-0.062872,-0.065233,1.000000,-0.354839,-0.186167,-0.216769
NO2,0.069309,0.000950,-0.354839,1.000000,0.544807,0.427693
PM10,0.167238,-0.018252,-0.186167,0.544807,1.000000,0.600095
PM25,0.067437,-0.017690,-0.216769,0.427693,0.600095,1.000000


In [ ]:
gg=newData.groupby(level=0)
sites=np.array([])
for n,g in gg:
    sites=np.append(sites,int(n))

In [ ]:
np.save('models/sites.npy',sites)

In [ ]:
data.isnull().sum()

In [20]:
newDataSet=None
groups=newData.groupby(level=0)
i=0
for name,g in groups:
    gg=g.reset_index().set_index('date').resample('D').mean().reset_index().set_index(['idPolair','date'])
    print(gg.shape)
    if(i==0):
        newDataSet=gg
    else:
        newDataSet=pd.concat([newDataSet,gg])
    i=i+1

(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1828, 6)
(1827, 6)


In [42]:
newDataSet=pd.read_csv('fillna/finalDays.csv')
newDataSet['date']=pd.to_datetime(newDataSet['date'])
newDataSet=newDataSet.set_index(['idPolair','date'])


In [43]:
x=pd.merge(sites,newDataSet,left_index=True,right_index=True)

In [44]:
x.head()

coord_x_l93  coord_y_l93         O3       NO2       PM10  \
idPolair date                                                                   
7001     2012-01-01    703667.89   6518340.92  41.375000  5.666667  14.292423   
         2012-01-02    703667.89   6518340.92  59.458333  2.041667   9.083154   
         2012-01-03    703667.89   6518340.92  61.625000  4.333333  13.969115   
         2012-01-04    703667.89   6518340.92  62.041667  7.041667   9.521076   
         2012-01-05    703667.89   6518340.92  68.083333  0.671022  10.884228   

                         PM25  
idPolair date                  
7001     2012-01-01  8.693054  
         2012-01-02  5.342532  
         2012-01-03  6.073384  
         2012-01-04  5.920439  
         2012-01-05  6.302409

In [8]:
models=dict({})

In [45]:
groups=x.groupby(level=0)

In [53]:
model = MultiOutputRegressor(estimator=RandomForestRegressor(random_state=0,n_estimators=10,max_depth=10))

In [54]:
xx=x.iloc[:,2:]
Xy=pd.concat([x,x.shift(-1),x.shift(-2),x.shift(-3),x.shift(-4),x.shift(-5),x.shift(-6),x.shift(-7),x.shift(-8),x.shift(-9),xx.shift(-10),xx.shift(-11),xx.shift(-12)],keys=['t','t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12'],axis=1).dropna()
X=Xy.iloc[:,:60]
y=Xy.iloc[:,60:]
In=X.values
Out=y.values

In [55]:
model.fit(In,Out)

KeyboardInterrupt: 

In [33]:
filename = 'bigForest.model'
pickle.dump(model, open(filename, 'wb'))

In [ ]:

for name,x in groups:
    Xy=pd.concat([x,x.shift(-1),x.shift(-2),x.shift(-3),x.shift(-4),x.shift(-5),x.shift(-6),x.shift(-7),x.shift(-8),x.shift(-9),x.shift(-10),x.shift(-11),x.shift(-12)],keys=['t','t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12'],axis=1).dropna()
    X=Xy.iloc[:,:40]
    y=Xy.iloc[:,40:]
    In=X.values.reshape(X.shape[0],10,4)
    Out=y.values.reshape(y.shape[0],12)
    model = models[name]=model
    model.reset_states()
    model.fit(In,Out,verbose=2,epochs=100)
    models[name]=model
    break

In [34]:
model

In [ ]:
newDataSet.shift(-1).head()

In [ ]:
import time

In [ ]:
x=time.time()

In [ ]:
y=time.time()
y-x

In [ ]:
def load_models(dir='models/'):
    x=time.time()
    sites=np.load(dir+'sites.npy')
    models=dict({})
    for i in sites:
        print(i)
        models[i]=load_model(dir+str(int(i))+'.model')
    y=time.time()
    print((y-x))
    return models

In [ ]:
models=load_models()

In [ ]:
models

In [ ]:
K.clear_session()

In [ ]:
x=time.time()
m=load_model('models/33111.model')
y=time.time()

In [ ]:
x1=time.time()
m1=load_model('models/33111.model',compile=False,)
y1=time.time()

In [ ]:
y-x

In [ ]:
y1-x1

In [ ]:
m.save_weights('models/w.test')

In [ ]:
m1.weights

In [ ]:
mo = Sequential()
mo.add(LSTM(200,input_shape=(4,10),dropout=0.4))
mo.add(Dense(12))
mo.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
x2=time.time()
mo.load_weights('models/w.test')
y2=time.time()

In [ ]:
y2-x2

In [ ]:
newDataSet=pd.read_csv('fillna/finalDays.csv')

In [ ]:
newDataSet.set_index(['idPolair','date'],inplace=True)

In [ ]:
groups=newDataSet.groupby(level=0)

In [ ]:
mmm=dict({})

for name,x in groups:
    Xy=pd.concat([x,x.shift(-1),x.shift(-2),x.shift(-3),x.shift(-4),x.shift(-5),x.shift(-6),x.shift(-7),x.shift(-8),x.shift(-9),x.shift(-10),x.shift(-11),x.shift(-12)],keys=['t','t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12'],axis=1).dropna()
    X=Xy.iloc[:,:40]
    y=Xy.iloc[:,40:]
    In=X.values.reshape(X.shape[0],4,10)
    Out=y.values.reshape(y.shape[0],12)
    model = Sequential()
    model.add(LSTM(20,input_shape=(4,10),dropout=0.4))
    model.add(Dense(12))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(In,Out,verbose=2,epochs=100)
    mmm[name]=model
    mmm[name].save('models2/'+str(int(name))+'.model')

In [ ]:
x2=time.time()
m2=load_model('models2/7001.model')
y2=time.time()

In [ ]:
y2-x2

In [ ]:
x2=time.time()
m2=load_model('models2/7001.model',compile=False)
y2=time.time()

In [ ]:
y2-x2

In [ ]:
import h5py

In [ ]:
def load_models2(dir='models/'):
    x=time.time()
    sites=np.load(dir+'sites.npy')
    models=dict({})
    for i in sites:
        K.clear_session()
        print(i)
        models[i]=load_model(dir+str(int(i))+'.model')
    y=time.time()
    print((y-x))
    return models

In [ ]:
momo=load_models2()

In [42]:
x=utils.load_data()

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


In [43]:
x.keys()

dict_keys(['sites', 'chimeres', 'geops', 'meteo', 'concentrations'])

In [ ]:
y=utils.get_data_day(10,x)

In [ ]:
gg=y[0]['NO2'].set_index(['idPolair','date']).sort_index().groupby(level=0)

In [ ]:
for i,j in gg:
    print(j)
    break

In [74]:
def run_predict(year=2016, max_days=10, dirname="../Data/training", list_days=None):
    """
    year : year to be evaluated
    max_days: number of past days allowed to predict a given day (set to 10 on the platform)
    dirname: path to the dataset
    list_days: list of days to be evaluated (if None the full year is evaluated)
    """

    overall_start = time.time()  # <== Mark starting time
    data = load_data(year=year, dirname=dirname)  # load all data files
    sites = data["sites"]  # get sites info
    day_results = dict({})
    if list_days is None:
        if calendar.isleap(year):  # check if year is leap
            list_days = range(366)
        else:
            list_days = range(365)
    for day in list_days:
        print(day)
        return  get_data_day(day, data, max_days=max_days,
                                                                              year=year)  # you will get an extraction of the year datasets, limited to the past max_days for each day
        day_results[day] = predict(day, sites, chimeres_day, geops_day, meteo_day,
                                   concentrations_day)  # do the prediction

    overall_time_spent = time.time() - overall_start  # end computation time
    pickle.dump(day_results, open('submission/results.pk', 'wb'))  # save results
    pickle.dump(overall_time_spent, open('submission/time.pk', 'wb'))  # save computation time

In [75]:
chimeres_day, geops_day, meteo_day, concentrations_day =run_predict(list_days=[14])

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


14


In [ ]:
c=concentrations_day

In [ ]:
c.keys()

In [84]:
def toPredcit(concentrations_day,sites):
    sites['idPolair']=pd.to_numeric(sites['idPolair'])
    sites=sites.set_index('idPolair').loc[:,['coord_x_l93','coord_y_l93']]/10000
    O3=pd.DataFrame(concentrations_day['O3'].set_index(['idPolair','date'])['Valeur'])
    O3.columns=['O3']
    NO2=pd.DataFrame(concentrations_day['NO2'].set_index(['idPolair','date'])['Valeur'])
    NO2.columns=['NO2']
    PM10=pd.DataFrame(concentrations_day['PM10'].set_index(['idPolair','date'])['Valeur'])
    PM10.columns=['PM10']
    PM25=pd.DataFrame(concentrations_day['PM25'].set_index(['idPolair','date'])['Valeur'])
    PM25.columns=['PM25']
    cons=pd.merge(O3,pd.merge(NO2,pd.merge(PM10,PM25,left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer').reset_index()
    cons['idPolair']=pd.to_numeric(cons['idPolair'])
    cons['date']=pd.to_datetime(cons['date'])
    cons=cons.set_index('idPolair')
    X=pd.merge(sites,cons,left_index=True,right_index=True)
    X=X.reset_index()
    X=X.set_index(['idPolair','date']).sort_index()
    i=0

    means=X.reset_index().set_index('date').resample('D').mean()
    del means['idPolair']
    meanOfmeans=pd.DataFrame((np.array(means.mean().tolist()).reshape(1,6)),columns=['X','Y','O3', 'NO2', 'PM10', 'PM25'])
    meanOfmeans=pd.concat([meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans],axis=1)
    newX=None
    groups=X.groupby(level=0)
    i=0
    for name,g in groups:
        gg=g.reset_index().set_index('date').resample('D').mean().reset_index().set_index(['idPolair','date']).fillna(method='ffill').fillna(method='bfill').fillna(means).fillna(0)
        shape=gg.shape[0]
        gg=pd.DataFrame(np.array(pd.concat([gg.shift(10),gg.shift(9),gg.shift(8),gg.shift(7),gg.shift(6),gg.shift(5),gg.shift(4),gg.shift(3),gg.shift(2),gg.shift(1)],axis=1).iloc[shape-1,:].tolist()).reshape(1,60),index=[name],columns=['X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25','X','Y','O3', 'NO2', 'PM10', 'PM25'])
        meanOfmeans.index=[name]
        gg=gg.fillna(meanOfmeans)
        if(i==0):
            newX=gg
        else:
            newX=pd.concat([newX,gg])
        i=i+1
    return newX

In [85]:
newX=toPredcit(concentrations_day,x['sites'])

In [86]:
newX

,X,Y,O3,NO2,PM10,PM25,X,Y,O3,NO2,...,O3,NO2,PM10,PM25,X,Y,O3,NO2,PM10,PM25
7001,70.366789,651.834092,76.958333,3.708333,11.620183,7.926931,70.366789,651.834092,55.416667,8.500000,...,63.000000,6.375000,15.852232,8.367089,70.366789,651.834092,45.000000,15.916667,17.225420,14.436170
7004,70.884203,652.196805,67.583333,10.916667,4.750000,4.000000,70.884203,652.196805,47.375000,18.416667,...,51.000000,19.958333,12.000000,5.000000,70.884203,652.196805,32.416667,27.500000,10.045455,5.000000
7009,70.679922,651.917674,68.916667,12.583333,5.083333,7.926931,70.679922,651.917674,50.166667,18.666667,...,51.416667,22.250000,12.666667,8.367089,70.679922,651.917674,35.041667,32.125000,10.208333,14.436170
7010,66.909266,658.272251,43.671194,14.916667,8.666667,7.926931,66.909266,658.272251,32.159854,20.625000,...,44.970440,25.750000,16.875000,8.367089,66.909266,658.272251,28.386628,29.291667,12.625000,14.436170
7011,65.554787,642.505177,43.671194,30.254881,11.620183,7.926931,65.554787,642.505177,32.159854,32.746531,...,44.970440,33.372130,15.852232,8.367089,65.554787,642.505177,28.386628,41.351121,17.225420,14.436170
7012,76.949688,643.877414,43.671194,30.254881,11.620183,7.926931,76.949688,643.877414,32.159854,32.746531,...,44.970440,33.372130,15.852232,8.367089,76.949688,643.877414,28.386628,41.351121,17.225420,14.436170
7013,66.958584,658.231669,65.333333,8.875000,11.620183,7.926931,66.958584,658.231669,48.541667,12.958333,...,47.250000,17.041667,15.852232,8.367089,66.958584,658.231669,32.166667,21.333333,17.225420,14.436170
7014,65.422892,642.229442,43.671194,30.254881,11.620183,7.926931,65.422892,642.229442,32.159854,32.746531,...,44.970440,33.372130,15.852232,8.367089,65.422892,642.229442,28.386628,41.351121,17.225420,14.436170
7015,69.724646,651.916641,80.666667,30.254881,11.620183,7.926931,69.724646,651.916641,69.083333,32.746531,...,75.291667,33.372130,15.852232,8.367089,69.724646,651.916641,73.833333,41.351121,17.225420,14.436170
7016,66.932515,658.054734,66.250000,30.254881,11.620183,7.926931,66.932515,658.054734,50.833333,32.746531,...,50.375000,33.372130,15.852232,8.367089,66.932515,658.054734,35.250000,41.351121,17.225420,14.436170


In [44]:
newX.iloc[:5,7*4:]

,O3,NO2,PM10,PM25,O3,NO2,PM10,PM25,O3,NO2,PM10,PM25
7001,78.041667,1.291667,7.948734,4.494505,63.000000,6.375000,15.852232,8.367089,45.000000,15.916667,17.225420,14.43617
7004,68.041667,8.166667,3.791667,5.000000,51.000000,19.958333,12.000000,5.000000,32.416667,27.500000,10.045455,5.00000
7009,72.125000,7.708333,4.500000,4.494505,51.416667,22.250000,12.666667,8.367089,35.041667,32.125000,10.208333,14.43617
7010,55.291966,16.727273,11.916667,4.494505,44.970440,25.750000,16.875000,8.367089,28.386628,29.291667,12.625000,14.43617
7011,55.291966,23.875803,7.948734,4.494505,44.970440,33.372130,15.852232,8.367089,28.386628,41.351121,17.225420,14.43617


In [47]:
yy['NO2']

{'7001': {'D0': array([12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726,
         12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726,
         12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726,
         12.664726, 12.664726, 12.664726, 12.664726, 12.664726, 12.664726],
        dtype=float32),
  'D1': array([16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081,
         16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081,
         16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081,
         16.599081, 16.599081, 16.599081, 16.599081, 16.599081, 16.599081],
        dtype=float32),
  'D2': array([17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357,
         17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357,
         17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357,
         17.267357, 17.267357, 17.267357, 17.267357, 17.267357, 17.267357],
        dtype=float32)}

In [91]:
model_lkl=load_model('lklMarraWe7di.model')

def predict(X,model,dir='models/'):
    
    #define a dictionary to return the result
    results = dict({})
    
    #add the 4 pol to the dict
    for pol in ["PM10", "PM25", "O3", "NO2"]:
        results[pol] = dict({})
        #add each idPolair to each pol in the dict
        sites=np.load('models/sites.npy')
        for idPolair in sites:
            results[pol][''+str(int(idPolair))] = dict({})
        
    
    #for each pol predict and add the result
    for index,row in X.iterrows():
        
        
        #predict the 12 values
        y_predict = model.predict(row.values.reshape(1,10,6))
        
        #assign each value to the right place
        idPolair=str(int(index))
        print(idPolair)
        results["NO2"][idPolair]["D0"] = np.full((24), y_predict[0, 1])
        results["NO2"][idPolair]["D1"] = np.full((24),y_predict[0, 5])
        results["NO2"][idPolair]["D2"] = np.full((24),y_predict[0, 9])
        results["O3"][idPolair]["D0"] = np.full((24),y_predict[0, 0])
        results["O3"][idPolair]["D1"] = np.full((24),y_predict[0, 4])
        results["O3"][idPolair]["D2"] = np.full((24),y_predict[0, 8])
        results["PM10"][idPolair]["D0"] = np.full((24),y_predict[0, 2])
        results["PM10"][idPolair]["D1"] = np.full((24),y_predict[0, 6])
        results["PM10"][idPolair]["D2"] = np.full((24),y_predict[0, 10])
        results["PM25"][idPolair]["D0"] = np.full((24),y_predict[0, 3])
        results["PM25"][idPolair]["D1"] = np.full((24),y_predict[0, 7])
        results["PM25"][idPolair]["D2"] = np.full((24),y_predict[0, 11])
        
    return results
        
        
        
        

In [92]:
yy=predict(newX,model_lkl)

7001
7004
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7020
7022
7028
7029
7031
7032
7034
7039
7042
7043
7045
7048
7049
7051
7052
7053
7054
7056
7057
7058
15013
15017
15018
15031
15038
15039
15043
15045
15046
15048
15049
20004
20013
20017
20019
20029
20031
20037
20045
20046
20047
20048
20049
20061
20062
20063
20065
20069
20070
27002
27003
27004
27005
27007
27008
27010
29421
29423
29424
29426
29428
29429
29439
29440
29441
33101
33102
33105
33111
33114
33120
33121
33122
33201
33202
33203
33211
33212
33213
33220
33232
33233
33235
33302
33303
33305
33367
33374
33414
36001
36002
36003
36005
36019
36021


In [ ]:



for idPolair in sites.idPolair:
    if int(idPolair) in X:
        y_predict = model.predict(np.array([X[int(idPolair)]]))
        results["NO2"][idPolair]["D0"] = np.full((24), y_predict[0, 0])
        results["NO2"][idPolair]["D1"] = np.full((24),y_predict[0, 4])
        results["NO2"][idPolair]["D2"] = np.full((24),y_predict[0, 8])
        results["O3"][idPolair]["D0"] = np.full((24),y_predict[0, 1])
        results["O3"][idPolair]["D1"] = np.full((24),y_predict[0, 5])
        results["O3"][idPolair]["D2"] = np.full((24),y_predict[0, 9])
        results["PM10"][idPolair]["D0"] = np.full((24),y_predict[0, 2])
        results["PM10"][idPolair]["D1"] = np.full((24),y_predict[0, 6])
        results["PM10"][idPolair]["D2"] = np.full((24),y_predict[0, 10])
        results["PM25"][idPolair]["D0"] = np.full((24),y_predict[0, 3])
        results["PM25"][idPolair]["D1"] = np.full((24),y_predict[0, 7])
        results["PM25"][idPolair]["D2"] = np.full((24),y_predict[0, 11])
    else:

In [93]:
yy

{'PM10': {'7001': {'D0': array([8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937,
          8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937,
          8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937,
          8.914937, 8.914937, 8.914937, 8.914937, 8.914937, 8.914937],
         dtype=float32),
   'D1': array([8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651,
          8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651,
          8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651,
          8.734651, 8.734651, 8.734651, 8.734651, 8.734651, 8.734651],
         dtype=float32),
   'D2': array([8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294,
          8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294,
          8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294,
          8.626294, 8.626294, 8.626294, 8.626294, 8.626294, 8.626294],
         dtype=float32)},
  '7004': {'D0': array([10.453365, 10.453365, 1

In [37]:
sites.loc[:,['coord_x_l93','coord_y_l93']]

,coord_x_l93,coord_y_l93
idPolair,,
15013,91.466844,644.596850
15017,91.097088,645.816550
15018,90.269900,647.672600
15031,97.331581,643.921300
15038,91.620200,645.751600
15039,91.241138,645.465050
15043,91.491600,645.510200
15045,91.067863,644.343650
15046,91.365000,645.717500


In [98]:
import p

In [99]:
p.run_predict()

INFO:utils:Loading sites files
INFO:utils:Loading Chimere files
INFO:utils:Loading Geop files
INFO:utils:Site 33374 does not have a geop file
INFO:utils:Loading Meteo files
INFO:utils:Loading concentrations measure files


0
2.169496536254883
1
2.0750174522399902
2
2.1517012119293213
3
2.4328041076660156
4
2.3769967555999756
5
2.6255435943603516
6
2.7225472927093506
7
3.2900400161743164
8
3.1982760429382324
9
3.336198568344116
10
3.463796615600586
11
3.0654845237731934
12
3.8391804695129395
13
3.255401134490967
14
3.319997787475586
15
3.0552196502685547
16
3.3727381229400635
17
3.9056129455566406
18
3.833986282348633
19
3.608930826187134
20
3.921595335006714
21
3.736147880554199
22
3.403850555419922
23
3.0764482021331787
24
3.6064016819000244
25
3.739269256591797
26
3.5093483924865723
27
3.1874783039093018
28
3.9500675201416016
29
4.05430793762207
30
4.233757257461548
31
3.9810845851898193
32
3.5930826663970947
33
3.0700342655181885
34
3.733464002609253
35
3.0650670528411865
36
3.278904438018799
37
3.073699712753296
38
3.8197226524353027
39
3.4166862964630127
40
3.414902925491333
41
3.101870059967041
42
3.2647945880889893
43
3.086899995803833
44
3.809231996536255
45
3.3476297855377197
46
3.50440907478332

In [103]:
import scoring

In [102]:
del scoring

In [105]:
model.reset_states()

In [107]:
from test import pr

ModuleNotFoundError: No module named 'test.prediction'; 'test' is not a package

In [ ]:
from utils import load_data, get_data_day
import numpy as np
import pandas as pd

import pickle
import datetime
import calendar
import time
import os

import utils
import logging
import pickle


logging.basicConfig(level=logging.INFO)

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.models import load_model
from keras import backend as K



dirname = os.path.dirname(__file__)

"""

Author: Hussein Awala

Models: Keras LSTM for each site taking the concentrations for the previous 10 days as input and predict the concentration for the 3 next days.

"""



def predict(day, sites, chimeres_day, geops_day, meteo_day, concentrations_day, model=None):

    start=time.time()

    X=toPredcit(concentrations_day)
    
    #define a dictionary to return the result
    results = dict({})
    
    #add the 4 pol to the dict
    for pol in ["PM10", "PM25", "O3", "NO2"]:
        results[pol] = dict({})
        #add each idPolair to each pol in the dict
        #sites=np.load('models/sites.npy')
        for idPolair in sites.idPolair:
            results[pol][''+str(int(idPolair))] = dict({})
        
    #results[pol]['33374'] = dict({})
    
    #for each pol predict and add the result
    for index,row in X.iterrows():
        
        #remove old model from GPU to speed up the model
        K.clear_session()
        
        #load the model of this site
        filename = os.path.join(dirname,'models/'+ str(int(index))+'.model')
        model=load_model(filename)
        
        #predict the 12 values
        y_predict = model.predict(row.values.reshape(1,10,4))
        
        #assign each value to the right place
        idPolair=str(int(index))
        results["NO2"][idPolair]["D0"] = np.full((24), y_predict[0, 1])
        results["NO2"][idPolair]["D1"] = np.full((24),y_predict[0, 5])
        results["NO2"][idPolair]["D2"] = np.full((24),y_predict[0, 9])
        results["O3"][idPolair]["D0"] = np.full((24),y_predict[0, 0])
        results["O3"][idPolair]["D1"] = np.full((24),y_predict[0, 4])
        results["O3"][idPolair]["D2"] = np.full((24),y_predict[0, 8])
        results["PM10"][idPolair]["D0"] = np.full((24),y_predict[0, 2])
        results["PM10"][idPolair]["D1"] = np.full((24),y_predict[0, 6])
        results["PM10"][idPolair]["D2"] = np.full((24),y_predict[0, 10])
        results["PM25"][idPolair]["D0"] = np.full((24),y_predict[0, 3])
        results["PM25"][idPolair]["D1"] = np.full((24),y_predict[0, 7])
        results["PM25"][idPolair]["D2"] = np.full((24),y_predict[0, 11])
    end=time.time()
    print((end-start))
    return results
        
        
        
        


def run_predict(year=2016, max_days=10, dirname="../Data/training", list_days=None):
    """
    year : year to be evaluated
    max_days: number of past days allowed to predict a given day (set to 10 on the platform)
    dirname: path to the dataset
    list_days: list of days to be evaluated (if None the full year is evaluated)
    """

    overall_start = time.time()  # <== Mark starting time
    data = load_data(year=year, dirname=dirname)  # load all data files
    sites = data["sites"]  # get sites info
    day_results = dict({})
    if list_days is None:
        if calendar.isleap(year):  # check if year is leap
            list_days = range(366)
        else:
            list_days = range(365)
    for day in list_days:
        print(day)
        chimeres_day, geops_day, meteo_day, concentrations_day = get_data_day(day, data, max_days=max_days,
                                                                              year=year)  # you will get an extraction of the year datasets, limited to the past max_days for each day
        day_results[day] = predict(day, sites, chimeres_day, geops_day, meteo_day,
                                   concentrations_day)  # do the prediction

    overall_time_spent = time.time() - overall_start  # end computation time
    pickle.dump(day_results, open('submission/results.pk', 'wb'))  # save results
    pickle.dump(overall_time_spent, open('submission/time.pk', 'wb'))  # save computation time



def toPredcit(concentrations_day):
    O3=pd.DataFrame(concentrations_day['O3'].set_index(['idPolair','date'])['Valeur'])
    O3.columns=['O3']
    NO2=pd.DataFrame(concentrations_day['NO2'].set_index(['idPolair','date'])['Valeur'])
    NO2.columns=['NO2']
    PM10=pd.DataFrame(concentrations_day['PM10'].set_index(['idPolair','date'])['Valeur'])
    PM10.columns=['PM10']
    PM25=pd.DataFrame(concentrations_day['PM25'].set_index(['idPolair','date'])['Valeur'])
    PM25.columns=['PM25']
    X=pd.merge(O3,pd.merge(NO2,pd.merge(PM10,PM25,left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer'),left_index=True,right_index=True,how='outer')
    X=X.reset_index()
    X['idPolair']=pd.to_numeric(X['idPolair'])
    X['date']=pd.to_datetime(X['date'])
    X=X.set_index(['idPolair','date']).sort_index()
    i=0


    means=X.reset_index().set_index('date').resample('D').mean()
    del means['idPolair']
    meanOfmeans=pd.DataFrame((np.array(means.mean().tolist()).reshape(1,4)),columns=['O3', 'NO2', 'PM10', 'PM25'])
    meanOfmeans=pd.concat([meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans,meanOfmeans],axis=1)
    newX=None
    groups=X.groupby(level=0)
    i=0
    for name,g in groups:
        gg=g.reset_index().set_index('date').resample('D').mean().reset_index().set_index(['idPolair','date']).fillna(method='ffill').fillna(method='bfill').fillna(means).fillna(0)
        shape=gg.shape[0]
        gg=pd.DataFrame(np.array(pd.concat([gg.shift(10),gg.shift(9),gg.shift(8),gg.shift(7),gg.shift(6),gg.shift(5),gg.shift(4),gg.shift(3),gg.shift(2),gg.shift(1)],axis=1).iloc[shape-1,:].tolist()).reshape(1,40),index=[name],columns=['O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25','O3', 'NO2', 'PM10', 'PM25'])
        meanOfmeans.index=[name]
        gg=gg.fillna(meanOfmeans)
        if(i==0):
            newX=gg
        else:
            newX=pd.concat([newX,gg])
        i=i+1
    return newX